In [1]:
# Imports

# Primary Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import warnings
import datetime

# Dependency Imports
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from collections import Counter

# Script Imports
%matplotlib inline
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 100

# R4 Data Science Exercise
Author: Chaz Frazer

## Scope

#### Data Columns:
1. ID (user ID)
2. First_Play_Date (date user first used service)
3. Play_Date (date of play/stream)
4. Genre (genre of portal being played)

### Questions

#### Question A:
What are some of the first things you notice about this data set? What is in it?

#### Question B:
Using this data, please answer the question-  does genre usage shift week-to-week (relative to when a user first used the service)?  For example, are users more likely to play Fingerprint during week one of their subscription, but then Mashboard during week 2 and 3? 

#### Question C:
How would you improve the service by improving customer engagement? 

## Data Import

In [3]:
r4 = pd.read_csv('data/r4_data_exercise.csv', index_col = 0)